In [1]:
# Agent Integration - Combine RAG + Calendar + Gemini
PROJECT_ID = "avalon-conversational-agent"
BUCKET_NAME = f"{PROJECT_ID}-data"

from google.cloud import storage
import json

In [2]:
!pip install sentence-transformers chromadb google-api-python-client google-generativeai pytz python-dateutil -q

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.15.2 requires protobuf<7.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.
kfp-pipeline-spec 2.15.2 requires protobuf<7.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [3]:
# Load ChromaDB and RAG data
import chromadb
from sentence_transformers import SentenceTransformer

storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

# Load RAG documents
def load_json_from_gcs(blob_path):
    blob = bucket.blob(blob_path)
    return json.loads(blob.download_as_string())

office_info = load_json_from_gcs("raw/office_info.json")
providers = load_json_from_gcs("raw/providers.json")
services = load_json_from_gcs("raw/services.json")
faqs = load_json_from_gcs("raw/faqs.json")

print("Loaded RAG data from GCS")

Loaded RAG data from GCS


In [6]:
# Download ChromaDB from GCS
import os

os.makedirs("chroma_db", exist_ok=True)

blobs = bucket.list_blobs(prefix="chroma_db/")
for blob in blobs:
    if blob.name != "chroma_db/":
        local_path = blob.name.replace("chroma_db/", "chroma_db/")
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        blob.download_to_filename(local_path)
        print(f"Downloaded {blob.name}")

# Load it
import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(name="avalon_dental")

print(f"\nLoaded {collection.count()} chunks from GCS")

Downloaded chroma_db/chroma.sqlite3
Downloaded chroma_db/da1e400a-70f6-4786-b323-188aae8b36fe/data_level0.bin
Downloaded chroma_db/da1e400a-70f6-4786-b323-188aae8b36fe/header.bin
Downloaded chroma_db/da1e400a-70f6-4786-b323-188aae8b36fe/length.bin
Downloaded chroma_db/da1e400a-70f6-4786-b323-188aae8b36fe/link_lists.bin

Loaded 44 chunks from GCS


In [43]:
# Set up Google Calendar API
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timedelta
from dateutil import parser
import pytz

SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = 'avalon-conversational-agent-7570ac8b695d.json'
CALENDAR_ID = '29783fcaaaf3b125207f80638a051bdbbe856038631f21f724f8509f9d099cb3@group.calendar.google.com'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

est = pytz.timezone('US/Eastern')

print("Calendar API connected")

Calendar API connected


In [34]:
# Get available appointment slots by location
def get_available_slots(location=None, days_ahead=30, duration_minutes=60):
    # Location-specific hours
    if location == "Christiana":
        OFFICE_START_HOUR = 7
        OFFICE_START_MIN = 30
        OFFICE_END_HOUR = 18
        OFFICE_END_MIN = 30
    elif location == "Newport":
        OFFICE_START_HOUR = 8
        OFFICE_START_MIN = 0
        OFFICE_END_HOUR = 17
        OFFICE_END_MIN = 0
    else:
        # Default to overlap (both open)
        OFFICE_START_HOUR = 8
        OFFICE_START_MIN = 0
        OFFICE_END_HOUR = 17
        OFFICE_END_MIN = 0
    
    WORK_DAYS = [0, 1, 2, 3]  # Mon-Thu
    
    now = datetime.now(est)
    time_min = now.isoformat()
    time_max = (now + timedelta(days=days_ahead)).isoformat()
    
    events_result = service.events().list(
        calendarId=CALENDAR_ID,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    busy_periods = []
    for event in events:
        start_raw = event['start'].get('dateTime', event['start'].get('date'))
        end_raw = event['end'].get('dateTime', event['end'].get('date'))
        
        if 'T' in start_raw:
            busy_periods.append((parser.parse(start_raw), parser.parse(end_raw)))
        else:
            start_date = parser.parse(start_raw)
            busy_periods.append((
                est.localize(datetime.combine(start_date, datetime.min.time())),
                est.localize(datetime.combine(start_date + timedelta(days=1), datetime.min.time()))
            ))
    
    available = []
    current_day = now.date()
    
    for day_offset in range(days_ahead):
        check_date = current_day + timedelta(days=day_offset)
        
        if check_date.weekday() not in WORK_DAYS:
            continue
        
        # Generate slots in 15-min intervals
        slot_time = est.localize(datetime.combine(check_date, datetime.min.time().replace(hour=OFFICE_START_HOUR, minute=OFFICE_START_MIN)))
        end_time = est.localize(datetime.combine(check_date, datetime.min.time().replace(hour=OFFICE_END_HOUR, minute=OFFICE_END_MIN)))
        
        while slot_time + timedelta(minutes=duration_minutes) <= end_time:
            if slot_time > now:
                is_available = True
                slot_end = slot_time + timedelta(minutes=duration_minutes)
                for busy_start, busy_end in busy_periods:
                    if slot_time < busy_end and slot_end > busy_start:
                        is_available = False
                        break
                if is_available:
                    available.append(slot_time)
            slot_time += timedelta(minutes=15)
    
    return available

# Test both locations
print(f"Christiana slots: {len(get_available_slots('Christiana'))}")
print(f"Newport slots: {len(get_available_slots('Newport'))}")

Christiana slots: 606
Newport slots: 488


In [40]:
# Book an appointment with correct duration
def book_appointment(slot_time, patient_name, service_type, location, duration_minutes=60):
    event = {
        'summary': f'{service_type} - {patient_name}',
        'description': f'Patient: {patient_name}\nService: {service_type}\nLocation: {location}\nDuration: {duration_minutes} minutes',
        'start': {
            'dateTime': slot_time.isoformat(),
            'timeZone': 'US/Eastern',
        },
        'end': {
            'dateTime': (slot_time + timedelta(minutes=duration_minutes)).isoformat(),
            'timeZone': 'US/Eastern',
        },
    }
    
    event = service.events().insert(calendarId=CALENDAR_ID, body=event).execute()
    print(f"Booked: {patient_name} for {service_type} at {location}")
    print(f"Time: {slot_time.strftime('%A, %b %d at %I:%M %p')} ({duration_minutes} min)")
    return event

In [31]:
# Set up Gemini via Vertex AI
from vertexai.generative_models import GenerativeModel
import vertexai

vertexai.init(project=PROJECT_ID, location="us-central1")
gemini = GenerativeModel("gemini-2.0-flash-001")

print("Gemini ready")

Gemini ready


In [32]:
# Retrieve relevant context from RAG
def get_context(query, n_results=3):
    results = collection.query(query_texts=[query], n_results=n_results)
    return "\n".join(results["documents"][0])

# Test it
print(get_context("What are your hours?"))

Q: What are your hours? A: Christiana is open Monday-Thursday 7:30 AM to 6:30 PM. Newport is open Monday-Thursday 8:00 AM to 5:00 PM. Both locations are closed Friday-Sunday.
Q: What is your cancellation policy? A: We require 2 days (48 hours) notice for cancellations. This helps us offer the time slot to other patients and maintains your eligibility for the Rewards Program.
Braces Consultation: Evaluation for braces or clear aligners. Duration: 45 minutes. Cost: $0-$0. Complimentary consultation


In [33]:
# Conversational agent
def agent(user_message, conversation_history=[]):
    # Get relevant context from RAG
    context = get_context(user_message)
    
    # Get next few available slots
    slots = get_available_slots()[:5]
    availability = "\n".join([s.strftime('%A, %b %d at %I:%M %p') for s in slots])
    
    # Build prompt
    system_prompt = f"""You are a friendly scheduling assistant for Avalon Dental. 
You help patients with questions and booking appointments.

OFFICE INFORMATION:
{context}

NEXT AVAILABLE APPOINTMENTS:
{availability}

LOCATIONS:
- Christiana: 430 Christiana Medical Center, Newark DE (Mon-Thu 7:30 AM - 6:30 PM)
- Newport: 406 Larch Circle, Newport DE (Mon-Thu 8:00 AM - 5:00 PM)

RULES:
- Always ask which location (Christiana or Newport) when scheduling
- Be friendly and concise
- If asked about availability, show 2-3 options
- For booking, confirm: name, service, location, and time
"""
    
    # Build conversation
    messages = system_prompt + "\n\nConversation:\n"
    for msg in conversation_history:
        messages += f"{msg['role']}: {msg['content']}\n"
    messages += f"Patient: {user_message}\nAssistant:"
    
    # Generate response
    response = gemini.generate_content(messages)
    
    return response.text

# Test it
print(agent("Hi, I'd like to schedule a cleaning"))

Hi there! I can help with that. To schedule your cleaning, can you please tell me which location you prefer, Christiana or Newport?



In [35]:
# Test a conversation flow
history = []

def chat(message):
    global history
    response = agent(message, history)
    history.append({"role": "Patient", "content": message})
    history.append({"role": "Assistant", "content": response})
    print(f"Patient: {message}")
    print(f"Assistant: {response}\n")

chat("Hi, I'd like to schedule a cleaning")
chat("Newport please")
chat("Do you have anything Thursday morning?")

Patient: Hi, I'd like to schedule a cleaning
Assistant: Hi there! I can help with that. Which location is better for you, Christiana or Newport?


Patient: Newport please
Assistant: Okay! The next available appointments for a cleaning at our Newport location are:

*   Tuesday, December 16 at 09:45 AM
*   Tuesday, December 16 at 10:00 AM
*   Tuesday, December 16 at 10:15 AM

Does any of those times work for you? If not, I can check for other options.


Patient: Do you have anything Thursday morning?
Assistant: Yes, we do! Let me check availability for Thursday morning at our Newport location. We have:

*   Thursday, December 18 at 8:00 AM
*   Thursday, December 18 at 8:15 AM
*   Thursday, December 18 at 8:30 AM

Do any of those times work for you?




In [36]:
# Service durations from services.json
SERVICE_DURATIONS = {
    "cleaning": 45,
    "new patient exam": 60,
    "filling": 45,
    "crown": 90,
    "root canal": 90,
    "extraction": 30,
    "whitening": 60,
    "emergency": 30,
    "consultation": 45,
    "implant": 90,
    "braces consultation": 45,
}

def get_duration(service_type):
    service_lower = service_type.lower()
    for key, duration in SERVICE_DURATIONS.items():
        if key in service_lower:
            return duration
    return 60  # default

# Conversational agent
def agent(user_message, conversation_history=[]):
    # Get relevant context from RAG
    context = get_context(user_message)
    
    # Detect service type from conversation
    full_convo = " ".join([m["content"] for m in conversation_history]) + " " + user_message
    service_type = "cleaning"  # default
    for s in SERVICE_DURATIONS.keys():
        if s in full_convo.lower():
            service_type = s
            break
    
    duration = get_duration(service_type)
    
    # Detect location from conversation
    location = None
    if "christiana" in full_convo.lower():
        location = "Christiana"
    elif "newport" in full_convo.lower():
        location = "Newport"
    
    # Get available slots with correct duration
    slots = get_available_slots(location=location, duration_minutes=duration)[:5]
    availability = "\n".join([s.strftime('%A, %b %d at %I:%M %p') for s in slots])
    
    # Build prompt
    system_prompt = f"""You are a friendly scheduling assistant for Avalon Dental. 
You help patients with questions and booking appointments.

OFFICE INFORMATION:
{context}

SERVICE BEING SCHEDULED: {service_type} ({duration} minutes)

NEXT AVAILABLE APPOINTMENTS:
{availability}

LOCATIONS:
- Christiana: 430 Christiana Medical Center, Newark DE (Mon-Thu 7:30 AM - 6:30 PM)
- Newport: 406 Larch Circle, Newport DE (Mon-Thu 8:00 AM - 5:00 PM)

RULES:
- Always ask which location (Christiana or Newport) when scheduling
- Be friendly and concise
- If asked about availability, show 2-3 options
- For booking, confirm: name, service, location, and time
"""
    
    # Build conversation
    messages = system_prompt + "\n\nConversation:\n"
    for msg in conversation_history:
        messages += f"{msg['role']}: {msg['content']}\n"
    messages += f"Patient: {user_message}\nAssistant:"
    
    # Generate response
    response = gemini.generate_content(messages)
    
    return response.text

print("Agent updated with service durations")

Agent updated with service durations


In [37]:
# Test conversation helper
history = []

def chat(message):
    global history
    response = agent(message, history)
    history.append({"role": "Patient", "content": message})
    history.append({"role": "Assistant", "content": response})
    print(f"Patient: {message}")
    print(f"Assistant: {response}\n")

def reset():
    global history
    history = []
    print("=" * 50)
    print("NEW CONVERSATION")
    print("=" * 50 + "\n")

# Test 1: Scheduling at Newport
reset()
chat("Hi I need a cleaning")
chat("Newport")
chat("Thursday morning?")

# Test 2: Scheduling at Christiana (longer procedure)
reset()
chat("I need a root canal")
chat("Christiana please")
chat("What's available Monday?")

# Test 3: Sunday appointments (should say no)
reset()
chat("Do you have any appointments on Sunday?")

# Test 4: RAG - doctor specialties
reset()
chat("What does Dr. Farhi specialize in?")

# Test 5: RAG - general info
reset()
chat("What's your cancellation policy?")

# Test 6: RAG - pricing
reset()
chat("How much does a crown cost?")

# Test 7: RAG - location info
reset()
chat("Where is the Christiana office located?")

# Test 8: RAG - insurance
reset()
chat("Do you accept insurance?")

NEW CONVERSATION

Patient: Hi I need a cleaning
Assistant: Hi there! I can help you with that. Which location would you prefer, Christiana or Newport?


Patient: Newport
Assistant: Okay! I see availability for a cleaning (45 minutes) at our Newport location on Monday, December 15th. 

Would any of these times work for you?
*   10:00 AM
*   10:15 AM
*   10:30 AM


Patient: Thursday morning?
Assistant: Okay, I can check for Thursday morning availability at our Newport location. What date are you interested in?


NEW CONVERSATION

Patient: I need a root canal
Assistant: Okay, I can help you with that! To schedule your root canal, which location would you prefer: Christiana or Newport?


Patient: Christiana please
Assistant: Great! At our Christiana location, I see availability for a root canal (90 minutes) on Monday, December 15th at:

*   10:00 AM
*   10:15 AM
*   10:30 AM

Do any of those times work for you?


Patient: What's available Monday?
Assistant: Okay! At our Christiana location

In [38]:
# Test: 45-minute gap on Monday 12/15 (10:00-10:45)
# Cleaning (45 min) should fit, root canal (90 min) should NOT

reset()
chat("I need a cleaning at Christiana on Monday between 10 and 11")

reset()
chat("I need a root canal at Christiana on Monday between 10 and 11")

NEW CONVERSATION

Patient: I need a cleaning at Christiana on Monday between 10 and 11
Assistant: Great! We have cleanings available at Christiana on Monday, December 15th at 10:00 AM. Does that time work for you? If not, we have another cleaning at 10:15 AM and 10:30 AM.
To confirm, can I please have your name?


NEW CONVERSATION

Patient: I need a root canal at Christiana on Monday between 10 and 11
Assistant: I don't have anything available at Christiana between 10 and 11 on Monday. I have appointments available on Monday, Dec 15 at 11:45 AM or 12:00 PM. Would either of those times work for you?




In [39]:
# Test booking an appointment
reset()
chat("I'd like to book a cleaning at Newport on Monday at 10:00 AM. My name is John Smith.")

# Actually book it
slots = get_available_slots(location="Newport", duration_minutes=45)
monday_10 = [s for s in slots if s.strftime('%A') == 'Monday' and s.strftime('%H:%M') == '10:00'][0]
book_appointment(monday_10, "John Smith", "Cleaning", "Newport")

NEW CONVERSATION

Patient: I'd like to book a cleaning at Newport on Monday at 10:00 AM. My name is John Smith.
Assistant: Okay John, I have you down for a cleaning at our Newport location on Monday, December 15th at 10:00 AM. Is that correct?


Booked: John Smith for Cleaning at Newport
Time: Monday, Dec 15 at 10:00 AM


{'kind': 'calendar#event',
 'etag': '"3531184524140446"',
 'id': '4fe9mjlorqbbg2r12tb2o3ngnc',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=NGZlOW1qbG9ycWJiZzJyMTJ0YjJvM25nbmMgMjk3ODNmY2FhYWYzYjEyNTIwN2Y4MDYzOGEwNTFiZGJiZTg1NjAzODYzMWYyMWY3MjRmODUwOWY5ZDA5OWNiM0Bn',
 'created': '2025-12-13T02:17:41.000Z',
 'updated': '2025-12-13T02:17:42.070Z',
 'summary': 'Cleaning - John Smith',
 'description': 'Patient: John Smith\nService: Cleaning\nLocation: Newport',
 'creator': {'email': 'calendar-agent@avalon-conversational-agent.iam.gserviceaccount.com'},
 'organizer': {'email': '29783fcaaaf3b125207f80638a051bdbbe856038631f21f724f8509f9d099cb3@group.calendar.google.com',
  'displayName': 'Avalon Dental Appointments',
  'self': True},
 'start': {'dateTime': '2025-12-15T10:00:00-05:00',
  'timeZone': 'America/New_York'},
 'end': {'dateTime': '2025-12-15T11:00:00-05:00',
  'timeZone': 'America/New_York'},
 'iCalUID': '4fe9mjlorqbbg2r12tb2o3ngnc@google.com',
 'se

In [44]:
# Test booking with correct duration
slots = get_available_slots(location="Newport", duration_minutes=45)
slot_time = slots[0]  # First available slot

duration = get_duration("cleaning")  # 45 minutes
book_appointment(slot_time, "Test Patient", "Cleaning", "Newport", duration)

print(f"\nCheck your calendar - should show {duration} minute appointment")

Booked: Test Patient for Cleaning at Newport
Time: Monday, Dec 15 at 10:00 AM (45 min)

Check your calendar - should show 45 minute appointment


In [45]:
# Book test appointments to screenshot
reset()

# Book a cleaning (45 min)
slots = get_available_slots(location="Newport", duration_minutes=45)
slot = slots[0]
book_appointment(slot, "Sarah Johnson", "Cleaning", "Newport", 45)

# Book a root canal (90 min) 
slots2 = get_available_slots(location="Christiana", duration_minutes=90)
slot2 = slots2[0]
book_appointment(slot2, "Mike Williams", "Root Canal", "Christiana", 90)

NEW CONVERSATION

Booked: Sarah Johnson for Cleaning at Newport
Time: Monday, Dec 15 at 10:00 AM (45 min)
Booked: Mike Williams for Root Canal at Christiana
Time: Monday, Dec 15 at 11:45 AM (90 min)


{'kind': 'calendar#event',
 'etag': '"3531185798468222"',
 'id': 'le4vhmt90c8ovpmohrit1undbs',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=bGU0dmhtdDkwYzhvdnBtb2hyaXQxdW5kYnMgMjk3ODNmY2FhYWYzYjEyNTIwN2Y4MDYzOGEwNTFiZGJiZTg1NjAzODYzMWYyMWY3MjRmODUwOWY5ZDA5OWNiM0Bn',
 'created': '2025-12-13T02:28:19.000Z',
 'updated': '2025-12-13T02:28:19.234Z',
 'summary': 'Root Canal - Mike Williams',
 'description': 'Patient: Mike Williams\nService: Root Canal\nLocation: Christiana\nDuration: 90 minutes',
 'creator': {'email': 'calendar-agent@avalon-conversational-agent.iam.gserviceaccount.com'},
 'organizer': {'email': '29783fcaaaf3b125207f80638a051bdbbe856038631f21f724f8509f9d099cb3@group.calendar.google.com',
  'displayName': 'Avalon Dental Appointments',
  'self': True},
 'start': {'dateTime': '2025-12-15T11:45:00-05:00',
  'timeZone': 'America/New_York'},
 'end': {'dateTime': '2025-12-15T13:15:00-05:00',
  'timeZone': 'America/New_York'},
 'iCalUID': 'le4vhmt90